In [3]:
from nuscenes.nuscenes import NuScenes
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import box_in_image, BoxVisibility
from pyquaternion import Quaternion
import numpy as np
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import importlib.util
import sys
import glob
import os
import importlib.util
import sys
import traceback

# Define the module name and file path
module_name = 'NuScenesExplorer'
file_path = 'nuscenes-devkit/python-sdk/nuscenes/nuscenes.py'
file_path = '/home/nuno120/Documents/SCRIPTIE/scri/lib/python3.10/site-packages/nuscenes/nuscenes.py'
try:
    # Load the module specification from the file location
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    if spec is None:
        raise ImportError(f"Cannot find module '{module_name}' at '{file_path}'")

    # Create a new module based on the specification
    NuScExpl = importlib.util.module_from_spec(spec)

    # Execute the module in its own namespace
    spec.loader.exec_module(NuScExpl)

    # Optionally, add the module to sys.modules
    sys.modules[module_name] = NuScExpl

    # Now you can use the imported module
    print(f"Module '{module_name}' loaded successfully from '{file_path}'")
except Exception as e:
    print(f"Error importing module '{module_name}' from '{file_path}': {e}")
    traceback.print_exc()

nusc = NuScenes(version='v1.0-mini', dataroot='/home/nuno120/Documents/SCRIPTIE/data/sets/nuscenes', verbose=True)




def get_sample_data(nusc_object, sample_data_token, box_vis_level=BoxVisibility.ANY, selected_anntokens=None):
    """
    Returns the data path as well as all annotations related to that sample_data(single image).
    Note that the boxes are transformed into the current sensor's coordinate frame.
    :param sample_data_token: <str>. Sample_data token(image token).
    :param box_vis_level: <BoxVisibility>. If sample_data is an image, this sets required visibility for boxes.
    :param selected_anntokens: [<str>]. If provided only return the selected annotation.
    :return: (data_path <str>, boxes [<Box>], camera_intrinsic <np.array: 3, 3>)
    """

    # Retrieve sensor & pose records
    sd_record = nusc_object.get('sample_data', sample_data_token)
    cs_record = nusc_object.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
    sensor_record = nusc_object.get('sensor', cs_record['sensor_token'])
    pose_record = nusc_object.get('ego_pose', sd_record['ego_pose_token'])

    sample_record = nusc_object.get('sample',sd_record['sample_token'])
    data_path = nusc_object.get_sample_data_path(sample_data_token)

    if sensor_record['modality'] == 'camera':
        cam_intrinsic = np.array(cs_record['camera_intrinsic'])
        imsize = (sd_record['width'], sd_record['height'])
    else:
        cam_intrinsic = None
        imsize = None

    # Retrieve all sample annotations and map to sensor coordinate system.
    if selected_anntokens is not None:
        boxes = list(map(nusc_object.get_box, selected_anntokens))
    else:
        boxes = nusc_object.get_boxes(sample_data_token)
        selected_anntokens = sample_record['anns']

    # Make list of Box objects including coord system transforms.
    box_list = []
    ann_list = []
    for box,ann in zip(boxes,selected_anntokens):

        # Move box to ego vehicle coord system
        box.translate(-np.array(pose_record['translation']))
        box.rotate(Quaternion(pose_record['rotation']).inverse)

        #  Move box to sensor coord system
        box.translate(-np.array(cs_record['translation']))
        box.rotate(Quaternion(cs_record['rotation']).inverse)

        if sensor_record['modality'] == 'camera' and not \
                box_in_image(box, cam_intrinsic, imsize, vis_level=box_vis_level):
            continue
        #print('ann', ann)

        annotation = nusc.get('sample_annotation', ann)
        #print(annotation)
        # Extract visibility level
        visibility_token = annotation['visibility_token']

        #print(visibility_token)
        if visibility_token == 1:
            continue
        #now get the visibility level from the ann
        #visibility = ann['visibility_token']
        
        

        box_list.append(box)
        ann_list.append(ann)
    #%matplotlib widget
    #ax =  plt.figure().add_subplot(111, projection='3d')
    #for box in box_list:
        #print('center: ', box.center)
        #print(box)
        #box.render(axis =ax)
    #ax.scatter(0, 0, 0, c='g', marker='o')
    
    
    return data_path, box_list, ann_list, cam_intrinsic #single image info

import math 
def get_quaternion_from_euler(roll, pitch, yaw):
    qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    return [qx, qy, qz, qw]

# Function to convert YOLO format to bounding box coordinates
def convert_from_yolo_format(yolo_box, image_width, image_height):
    x_center = yolo_box[0]
    y_center = yolo_box[1]
    normalized_width = yolo_box[2]
    normalized_height = yolo_box[3]
    
    width = normalized_width * image_width
    height = normalized_height * image_height
    
    x_min = (x_center - normalized_width / 2) * image_width
    y_min = (y_center - normalized_height / 2) * image_height
    
    x_max = x_min + width
    y_max = y_min + height
    
    return round(x_min), round(y_min), round(x_max), round(y_max)

def threeD_2_twoD(boxsy,intrinsic): #input is a single annotation box
    '''
    given annotation boxes and intrinsic camera matrix
    outputs the 2d bounding box coordinates as a list (all annotations for a particular sample image)
    '''
    corners = boxsy.corners()
    x = corners[0,:]
    y = corners[1,:]
    z = corners[2,:]
    x_y_z = np.array((x,y,z))
    orthographic = np.dot(intrinsic,x_y_z)
    perspective_x = orthographic[0]/orthographic[2]
    perspective_y = orthographic[1]/orthographic[2]
    perspective_z = orthographic[2]/orthographic[2]
    
    min_x = np.min(perspective_x)
    max_x = np.max(perspective_x)
    min_y = np.min(perspective_y)
    max_y = np.max(perspective_y)
    

    
    return min_x,max_x,min_y,max_y

def all_3d_to_2d(boxes,anns,intrinsic): #input 3d boxes, annotation key lists, intrinsic matrix (one image)
    x_min=[]
    x_max=[]
    y_min=[]
    y_max =[]
    width=[]
    height=[]
    objects_detected =[]
    orig_objects_detected =[]
    
   
    for j in range(len(boxes)): #iterate through boxes
        box=boxes[j]
        orig_objects_detected.append(box)
                
        center = box.center #get boxe's center

        center = np.dot(intrinsic,center)
        center_point = center/(center[2]) #convert center point into image plane
        
        
        if center_point[0] <-100 or center_point[0] > 1700 or center_point[1] <-100 or center_point[1] >1000:
            #if center of bounding box is outside of the image, do not annotate
            #pass
            min_x, max_x, min_y, max_y = threeD_2_twoD(box,intrinsic) #converts box into image plane
            w = max_x - min_x
            h = max_y - min_y


            x_min.append(min_x)
            x_max.append(max_x)
            y_min.append(min_y)
            y_max.append(max_y)
            width.append(w)
            height.append(h)
            objects_detected.append(box)
            
        
        else:
            min_x, max_x, min_y, max_y = threeD_2_twoD(box,intrinsic) #converts box into image plane
            w = max_x - min_x
            h = max_y - min_y


            x_min.append(min_x)
            x_max.append(max_x)
            y_min.append(min_y)
            y_max.append(max_y)
            width.append(w)
            height.append(h)
            objects_detected.append(box)
          

    return x_min,x_max,y_min,y_max,width,height,objects_detected,orig_objects_detected #for a single image

def make_grid(cropped_image, lidar_points, distances, shape=(10, 10)):
    """Generate a grid of LiDAR points and corresponding distances."""
    # Remove the third dimension from the lidar points
    lidar_points = lidar_points[:,:2]
    
    # Generate evenly distributed grid points
    x_coords = np.linspace(0, cropped_image.shape[1] - 1, shape[0])
    y_coords = np.linspace(0, cropped_image.shape[0] - 1, shape[1])
    grid_points_x, grid_points_y = np.meshgrid(x_coords, y_coords)

    grid_points = np.stack((grid_points_x.flatten(), grid_points_y.flatten()), axis=1)
    
    # Initialize arrays to store distances for grid points
    grid_distances = np.zeros(grid_points.shape[0])

    # Find closest grid point for each LiDAR point and assign the distance of the LiDAR point to that grid point
    for i, (x_lidar, y_lidar) in enumerate(lidar_points):
        # Calculate Euclidean distances between LiDAR point and all grid points
        distances_to_grid_points = np.sqrt((grid_points[:, 0] - x_lidar)**2 + (grid_points[:, 1] - y_lidar)**2)
        # Find index of closest grid point
        closest_grid_point_index = np.argmin(distances_to_grid_points)
        # Assign distance of LiDAR point to closest grid point
        grid_distances[closest_grid_point_index] = distances[i]


    #print('grid points shape:', grid_points.shape)
    #print('grid distances shape:', grid_distances.shape)
    return grid_points, grid_distances

def extract_bounding_box(i,camera_name): #give a single sample number and camera name
    
    '''
    input sample number i, camera name
    outputs min x, max x, min y max y, width and height of bounding box in image coordinates
    2d bounding box
    options for camera name : CAM_FRONT, CAM_FRONT_RIGHT, CAM_FRONT_LEFT, CAM_BACK, CAM_BACK_RIGHT,CAM_BACK_LEFT
    '''
    #nusc.sample[i] #one image
    
    camera_token = nusc.sample[i]['data']['%s' %camera_name] #one camera, get the camera token 

    #nusc.render_sample(nusc.sample[i]['token'])
    path, boxes, anns, intrinsic_matrix = get_sample_data(nusc,'%s' %camera_token) #gets data for one image
    
    return path, boxes, anns, intrinsic_matrix #info for a single image
    #x_min, x_max,y_min,y_max,width,height, objects_detected,orig_objects_detected = all_3d_to_2d(boxes,anns, intrinsic_matrix)
    #return x_min, x_max, y_min, y_max, width, height, path, boxes,intrinsic_matrix, objects_detected,orig_objects_detected,
    #info for a single image


def map_lidar_to_image(nusc, sample_number, camera, pointsensor_token, camera_token):
    explorer = NuScExpl.NuScenesExplorer(nusc)
    points, coloring, im = explorer.map_pointcloud_to_image(pointsensor_token, camera_token, min_dist=1, render_intensity=False, show_lidarseg=False, filter_lidarseg_labels=None, lidarseg_preds_bin_path=None, show_panoptic=False)
    #points, coloring, im = NuScExpl.map_pointcloud_to_image(pointsensor_token, camera_token, min_dist=0, render_intensity=False, show_lidarseg=False, filter_lidarseg_labels=None, lidarseg_preds_bin_path=None, show_panoptic=False)
    return points, coloring, im

import matplotlib.pyplot as plt
import numpy as np



def visualize_boxes_top_view(ax, box):
    '''
    input 3D box
    output ax with 3D box plotted in 2D (top view)
    '''
    print(box)
    x,y,z = box.center
    #plot the x and y 
    ax.scatter(x, z, c='b', marker='o', label= f"{x,z}", s=10)
    #add text with the center of the box
    ax.text(x, z, f"{int(x),int(z)}", fontsize=6)



    # Get the corners of the 3D box
    corners = box.corners()
    
    # Extract x, y coordinates for top view (ignore z)
    x = corners[0, :]
    y = corners[2, :]
    
    # Plot the top view of the box
    ax.plot([x[0], x[1]], [y[0], y[1]], 'r-') # Front side
    ax.plot([x[1], x[2]], [y[1], y[2]], 'r-') # Right side
    ax.plot([x[2], x[3]], [y[2], y[3]], 'r-') # Back side
    ax.plot([x[3], x[0]], [y[3], y[0]], 'r-') # Left side
    ax.plot([x[4], x[5]], [y[4], y[5]], 'r--') # Top front side
    ax.plot([x[5], x[6]], [y[5], y[6]], 'r--') # Top right side
    ax.plot([x[6], x[7]], [y[6], y[7]], 'r--') # Top back side
    ax.plot([x[7], x[4]], [y[7], y[4]], 'r--') # Top left side
    ax.plot([x[0], x[4]], [y[0], y[4]], 'r--') # Connection 1
    ax.plot([x[1], x[5]], [y[1], y[5]], 'r--') # Connection 2
    ax.plot([x[2], x[6]], [y[2], y[6]], 'r--') # Connection 3
    ax.plot([x[3], x[7]], [y[3], y[7]], 'r--') # Connection 4

    #add 0,0,0
    ax.scatter(0, 0, c='g', marker='o')



# Define a function to load the model with the necessary custom objects
def load_model_with_custom_objects(model_path):
    custom_objects = {
        "mse": MeanSquaredError()  # Map 'mse' to the MeanSquaredError loss function
    }
    return keras.models.load_model(model_path, custom_objects=custom_objects)


# Assuming `nusc` and related functions are already defined
i = 4
camera_names = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT', 'CAM_BACK_LEFT']

import json

def load_normalization_stats(file_path):
    with open(file_path, 'r') as file:
        stats_dict = json.load(file)
    return (stats_dict['distances_mean'], stats_dict['distances_std'],
            stats_dict['centers_x_mean'], stats_dict['centers_x_std'],
            stats_dict['centers_y_mean'], stats_dict['centers_y_std'],
            stats_dict['centers_z_mean'], stats_dict['centers_z_std'],
            stats_dict['sizes_x_mean'], stats_dict['sizes_x_std'],
            stats_dict['sizes_y_mean'], stats_dict['sizes_y_std'],
            stats_dict['sizes_z_mean'], stats_dict['sizes_z_std'],
            stats_dict['euler_angles_roll_mean'], stats_dict['euler_angles_roll_std'],
            stats_dict['euler_angles_pitch_mean'], stats_dict['euler_angles_pitch_std'],
            stats_dict['euler_angles_yaw_mean'], stats_dict['euler_angles_yaw_std'])


import cv2
import numpy as np
import matplotlib.pyplot as plt
#from nuscenes.nuscenes.utils.geometry_utils import Box
from pyquaternion import Quaternion

def get_training_data(image, boxes, anns, lidar_points, coloring, intrinsic_matrix):
    """print('points:', points)
    print('poinnts shape', points.shape)
    print('coloring:', coloring)
    print('coloring shape', coloring.shape)"""

    coordinate_points = []
    for i in range(len(coloring)):
        coordinates = lidar_points[:-1, i]
        [x, y] = coordinates
        distance = coloring[i]
        #print(x,y, distance)
        coordinate_points.append([x, y, distance])

    
    def croppad_image_lidar(im, coordinate_points, lidar_points, coloring, min_x, max_x, min_y, max_y, pixels):
        if im is None:
            return None, None,None, None, None, None, None
        image_width, image_height = im.size
        if min_x > pixels:
            min_x -= pixels
        if min_y > pixels:
            min_y -= pixels
        if max_x < image_width - pixels:
            max_x += pixels
        if max_y < image_height - pixels:
            max_y += pixels

        min_x = max(0, min_x)
        max_x = min(image_width, max_x)
        min_y = max(0, min_y)
        max_y = min(image_height, max_y)

        if min_x == max_x or min_y == max_y:
            return None, None, None, None, None, None, None

        points_in_box = []
        new_coloring = []
        points = lidar_points
        for j in range(len(points[0])):
            if min_x <= points[0][j] <= max_x and min_y <= points[1][j] <= max_y:
                points_in_box.append(points[:, j])
                new_coloring.append(coloring[j])

        points_in_box = np.array(points_in_box)

        cropped_image = np.array(im)[round(min_y):round(max_y), round(min_x):round(max_x)]
        #get the coordinates that are within the cropped image
        image_lidar_coords = []
        for [x,y,distance] in coordinate_points:
            if min_x <= x <= max_x and min_y <= y <= max_y:
                image_lidar_coords.append([x,y,distance])
        image_lidar_coords = np.array(image_lidar_coords)



        if cropped_image.size == 0:
            return None, None, None, None, None, None, None

        if len(points_in_box) > 0:
            points_in_box[:, 0] -= round(min_x)
            points_in_box[:, 1] -= round(min_y)

        #from new coloring make an array of 64 long.
        #shuffle the new_coloring first randomly
        np.random.shuffle(new_coloring)
        distances = np.zeros(64)
        for i in range(len(new_coloring)):
            if i < 64:
                distances[i] = new_coloring[i]

        new_coloring = distances
        



        
            #check if the distance is not 0

        """
        if len(points_in_box) > 0:
            points_in_box, new_coloring = make_grid(cropped_image, points_in_box, new_coloring, shape=(16, 16))
        else:
            #print('No points in box')
            #evenly divide the image into 16x16 grid over a 32x32 image
            x_coords = np.linspace(0, 32 - 1, 16)
            y_coords = np.linspace(0, 32 - 1, 16)
            grid_points_x, grid_points_y = np.meshgrid(x_coords, y_coords)
            points_in_box = np.stack((grid_points_x.flatten(), grid_points_y.flatten()), axis=1)
            #print('points in box shape:', points_in_box.shape)
            new_coloring = np.zeros(16 * 16)
        """
        
        #print(f"cropped_image.shape before resizing: {cropped_image.shape}")

        if cropped_image.shape[0] > cropped_image.shape[1]:
            new_height = 64
            new_width = max(1, int(cropped_image.shape[1] * (new_height / cropped_image.shape[0])))
            #print(f"Resizing to (new_width, new_height): ({new_width}, {new_height})")
            cropped_image = cv2.resize(cropped_image, (new_width, new_height), interpolation=cv2.INTER_AREA)
            pad = 64 - new_width
            left = pad // 2
            right = pad - left
            cropped_image = cv2.copyMakeBorder(cropped_image, 0, 0, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        else:
            new_width = 64
            new_height = max(1, int(cropped_image.shape[0] * (new_width / cropped_image.shape[1])))
            #print(f"Resizing to (new_width, new_height): ({new_width}, {new_height})")
            cropped_image = cv2.resize(cropped_image, (new_width, new_height), interpolation=cv2.INTER_AREA)
            pad = 64 - new_height
            top = pad // 2
            bottom = pad - top
            cropped_image = cv2.copyMakeBorder(cropped_image, top, bottom, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        print('distances shape finally, ', new_coloring.shape)
        return cropped_image, image_lidar_coords, new_coloring, min_x, max_x, min_y, max_y

    x_min, x_max, y_min, y_max, width, height, objects_detected, orig_objects_detected = all_3d_to_2d(boxes, anns, intrinsic_matrix)

    cropped_images = []
    distances = []
    new_xmin = []
    new_xmax = []
    new_ymin = []
    new_ymax = []
    new_objects_detected = []
    new_image_lidar_coords = []

    for i in range(len(x_min)):
        cropped_image, image_lidar_coords, new_coloring, min_x, max_x, min_y, max_y = croppad_image_lidar(image, coordinate_points, lidar_points, coloring, x_min[i], x_max[i], y_min[i], y_max[i], 0)
        if cropped_image is not None:
            cropped_images.append(cropped_image)
            distances.append(new_coloring)
            new_xmin.append(min_y)
            new_xmax.append(max_y)
            new_ymin.append(min_x)
            new_ymax.append(max_x)
            new_objects_detected.append(objects_detected[i])
            new_image_lidar_coords.append(image_lidar_coords)

    return cropped_images, new_image_lidar_coords, distances, new_xmin, new_xmax, new_ymin, new_ymax, new_objects_detected

   

def normalize(values, mean_val, std_val):
    return (values - mean_val) / std_val

def normalize_input(image, distance, box_2D, normalization_stats):
    distances_mean, distances_std, \
    centers_x_mean, centers_x_std, centers_y_mean, centers_y_std, centers_z_mean, centers_z_std, \
    sizes_x_mean, sizes_x_std, sizes_y_mean, sizes_y_std, sizes_z_mean, sizes_z_std, \
    euler_angles_roll_mean, euler_angles_roll_std, euler_angles_pitch_mean, euler_angles_pitch_std, euler_angles_yaw_mean, euler_angles_yaw_std = normalization_stats
    # Normalize distance
    normalized_distance = normalize(distance, distances_mean, distances_std)
    # Normalize bounding box coordinates
    """normalized_box_2D = np.zeros(8)
    normalized_box_2D[0] = normalize(box_2D[0], centers_x_mean, centers_x_std)
    normalized_box_2D[1] = normalize(box_2D[1], centers_y_mean, centers_y_std)
    normalized_box_2D[2] = normalize(box_2D[2], centers_z_mean, centers_z_std)
    normalized_box_2D[3] = normalize(box_2D[3], sizes_x_mean, sizes_x_std)
    normalized_box_2D[4] = normalize(box_2D[4], sizes_y_mean, sizes_y_std)
    normalized_box_2D[5] = normalize(box_2D[5], sizes_z_mean, sizes_z_std)
    normalized_box_2D[6] = normalize(box_2D[6], euler_angles_roll_mean, euler_angles_roll_std)
    normalized_box_2D[7] = normalize(box_2D[7], euler_angles_pitch_mean, euler_angles_pitch_std)
    normalized_box_2D[8] = normalize(box_2D[8], euler_angles_yaw_mean, euler_angles_yaw_std)"""
    
    # Normalize image (assuming it's already in [0, 1] range)
    normalized_image = image / 255
    
    return normalized_image, normalized_distance, box_2D


def denormalize_output(predictions, normalization_stats):
    _,_, centers_x_mean, centers_x_std, centers_y_mean, centers_y_std, centers_z_mean, centers_z_std, \
    sizes_x_mean, sizes_x_std, sizes_y_mean, sizes_y_std, sizes_z_mean, sizes_z_std, \
    euler_angles_roll_mean, euler_angles_roll_std, euler_angles_pitch_mean, euler_angles_pitch_std, euler_angles_yaw_mean, euler_angles_yaw_std = normalization_stats

    denormalized_predictions = np.zeros_like(predictions)
    denormalized_predictions[:, 3] = predictions[:, 3] * sizes_x_std + sizes_x_mean
    denormalized_predictions[:, 4] = predictions[:, 4] * sizes_y_std + sizes_y_mean
    denormalized_predictions[:, 5] = predictions[:, 5] * sizes_z_std + sizes_z_mean
    denormalized_predictions[:, 0] = predictions[:, 0] * centers_x_std + centers_x_mean
    denormalized_predictions[:, 1] = predictions[:, 1] * centers_y_std + centers_y_mean
    denormalized_predictions[:, 2] = predictions[:, 2] * centers_z_std + centers_z_mean
    denormalized_predictions[:, 6] = predictions[:, 6] * euler_angles_roll_std + euler_angles_roll_mean
    denormalized_predictions[:, 7] = predictions[:, 7] * euler_angles_pitch_std + euler_angles_pitch_mean
    denormalized_predictions[:, 8] = predictions[:, 8] * euler_angles_yaw_std + euler_angles_yaw_mean
    return denormalized_predictions

def run_yolo(image_path, imsz=1216, conf_threshold=0.7, weights_path="/home/nuno120/Downloads/results_DownlOAD/hva_exp13_imgs1200/epoch78.pt", visualize=False):
    image = cv2.imread(image_path)
    device = "cpu"
    latest_results = "/home/nuno120/Documents/SCRIPTIE/yolov9/results/latest_results"
    #make the directory
    os.system('mkdir /home/nuno120/Documents/SCRIPTIE/yolov9/results/latest_results')
    #empty the folder
    os.system('rm -rf /home/nuno120/Documents/SCRIPTIE/yolov9/results/latest_results')
    # Run YOLOv5 on the image
    image_width, image_height = image.shape[1], image.shape[0]
    !python /home/nuno120/Documents/SCRIPTIE/yolov9/detect_dual.py --weights {weights_path} --imgsz {imsz} --conf {conf_threshold} --source {image_path} --save-txt --save-conf --save-crop --project /home/nuno120/Documents/SCRIPTIE/yolov9/results --name latest_results --exist-ok 
    #open the latest_results/labels folder and find the file with the camera name in it
    classes, confidences, x_min, x_max, y_min, y_max= [],[],[],[],[],[]
    for filename in os.listdir(latest_results+'/labels'):
        with open(latest_results+'/labels/'+filename, 'r') as f:
            lines = f.readlines()
            for line in lines:
                #split on spaces
                line = line.split()
                category, x, y, w, h,conf = int(line[0]), float(line[1]), float(line[2]), float(line[3]), float(line[4]), float(line[5])
                #if category == 3:
                classes.append(category)
                confidences.append(conf)
                #now convert the boxes to xmin,ymin, xmax, ymax
                min_x, min_y, max_x, max_y = convert_from_yolo_format([x,y,w, h], image_width, image_height)
                #append
                x_min.append(min_x)
                y_min.append(min_y)
                x_max.append(max_x)
                y_max.append(max_y)

    if visualize:
        plt.clf()
        plt.imshow(image)
        for i in range(len(x_min)):
            plt.plot([x_min[i], x_max[i], x_max[i], x_min[i], x_min[i]], [y_min[i], y_min[i], y_max[i], y_max[i], y_min[i]], 'r-')
            #show the classes nnumber
            plt.text(x_min[i], y_min[i], classes[i], color='red', fontsize=8)
        plt.axis('off')
        plt.show()
        plt.clf()

    return min_x, min_y, max_x, max_y, conf, classes



def visualize_3DNN(boxes, image, cropped_images,distances, x_max, x_min, y_max, y_min, imagesandlidar = True):  
    """function to visualze the 3DNN predicted"""


    loaded_stats = load_normalization_stats('/home/nuno120/Documents/SCRIPTIE/normalization_stats_car.json')
    #visualize the 2d bounding boxes on the image
    plt.clf()
    plt.imshow(image)
    for i in range(len(x_min)):
        plt.plot([x_min[i], x_max[i], x_max[i], x_min[i], x_min[i]], [y_min[i], y_min[i], y_max[i], y_max[i], y_min[i]], 'r-')
        plt.text(x_min[i], y_min[i], objects_detected[i], color='red', fontsize=8)
    plt.axis('off')
    plt.show()
    plt.clf()
    points_in_box = []
    #for each box, get the lidar points and distances
    for i in range(len(x_min)):
            new_coloring = distances[i]
            cropped_image = cropped_images[i]
            if imagesandlidar:
                #visualize the cropped image and the lidar points
                plt.clf()
                plt.imshow(cropped_image)
                if len(points_in_box) > 0:
                    plt.scatter(points_in_box[:, 0], points_in_box[:, 1], c=new_coloring, s=1)
                    for idx, point in enumerate(points_in_box):
                        plt.text(point[0], point[1], str(int(new_coloring[idx])), color='red', fontsize=8)
                plt.axis('off')
                plt.show()
                plt.clf()
            #expand dimensions 
            """box_2D_input = np.expand_dims(np.array([float(x_min[i]), float(x_max[i]), float(y_min[i]), float(y_max[i])]), axis=0)
            cropped_image = np.expand_dims(cropped_image, axis=2)
            cropped_image = np.expand_dims(cropped_image, axis=0)
            new_coloring = np.expand_dims(new_coloring, axis=0)

            #run the neural network on the cropped image and new coloring
            model = load_model_with_custom_objects('/home/nuno120/Documents/SCRIPTIE/model_cardistallcams_epoch_18.keras')
            predictions = model.predict({'image_input': cropped_image, 'distances_input': new_coloring, 'box_2D_input': box_2D_input})

            #get the 3d bounding box 
            center = predictions[0][0:3]
            size = predictions[0][3:6]
            euler = predictions[0][6:9]"""            
            box_2D_input = np.array([x_min[i]/1600, x_max[i]/1600, y_min[i]/900, y_max[i]/900])
            # Assuming cropped_image, new_coloring, and box_2D_input are already defined
            # Normalize input
            print(new_coloring)
            cropped_image, normalized_distance, _ = normalize_input(cropped_image, new_coloring, box_2D_input, loaded_stats)
            print(normalized_distance)
            expanded_2D_box = np.expand_dims(box_2D_input, axis=0)

            cropped_image = np.expand_dims(cropped_image, axis=2)
            cropped_image  = np.expand_dims(cropped_image, axis=0)
            normalized_distance = np.expand_dims(normalized_distance, axis=0)

            # Run the neural network on the normalized input
            model = load_model_with_custom_objects('/home/nuno120/Documents/SCRIPTIE/model_carnormdbetter.keras')
            predictions = model.predict({'image_input': cropped_image, 'distances_input': normalized_distance, 'box_2D_input': expanded_2D_box})
            print('pred:, ', predictions)
            # Denormalize the predictions
            denormalized_predictions = denormalize_output(predictions, loaded_stats)
            print('denormalize_output', denormalized_predictions)
            # Get the 3D bounding box
            denormalized_predictions = denormalized_predictions[0]
            center = denormalized_predictions[:3]
            size = denormalized_predictions[3:6]
            euler = denormalized_predictions[6:]
    
            #convert euler to quaternion
            x,y,z,w = get_quaternion_from_euler(euler[0], euler[1], euler[2])
            orientation = Quaternion(w,x,y,z)

            predicted_box = Box(center, size, orientation, name=boxes[i].name, token=boxes[i].token)
            print('center predicted:', center)
            print('size predicted:', size)
            print('euler predicted:', euler)



            box = boxes[i]
            print('center actual:', box.center)
            print('size actual:', box.wlh)
            print('euler actual:', box.orientation)
            distance = math.sqrt((box.center[0])**2+(box.center[1])**2+(box.center[2])**2)
            error_distance_centers = math.sqrt((center[0]-box.center[0])**2+(center[1]-box.center[1])**2+(center[2]-box.center[2])**2)
            print('error distance centers:',error_distance_centers )

            #visualize the predicted box
            
            
            if distance < 50:
                fig, ax_2d = plt.subplots(figsize=(10, 10))
                visualize_boxes_top_view(ax_2d, predicted_box)
                plt.show()
                plt.clf()

                #now visualize them both using top view function
                fig, ax_2d = plt.subplots(figsize=(10, 10))
                visualize_boxes_top_view(ax_2d, box)
                plt.show()
                plt.clf()


def quaternion_to_euler(w, x, y, z):
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll = math.atan2(t0, t1)
    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch = math.asin(t2)
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw = math.atan2(t3, t4)
    return [roll, pitch, yaw]

"""
sample_number = 120

# Define the camera names
#camera_names = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT', 'CAM_BACK_LEFT']
#camera_names = ['CAM_FRONT_RIGHT']

# Iterate over the camera names
for camera_name in camera_names:
    # Get the data for the sample
    path, boxes, anns, intrinsic_matrix = extract_bounding_box(sample_number, camera_name)
    image = cv2.imread(path)
    print(image.shape)
    camera_token = nusc.sample[sample_number]['data']['%s' % camera_name]
    pointsensor_token = nusc.sample[sample_number]['data']['LIDAR_TOP']

    # Get the lidar points and coloring
    points, coloring, im = map_lidar_to_image(nusc, sample_number, camera_name, pointsensor_token, camera_token)

    #run yolo and show the boxes for comparison
    #min_x, min_y, max_x, max_y, conf, classes = run_yolo(path, conf_threshold=0.75,visualize=True)

    # Get the training data
    cropped_images, distances, x_min, x_max, y_min, y_max, objects_detected = get_training_data(im, boxes, anns, points, coloring, intrinsic_matrix)

    #visualize_3DNN(boxes, image, cropped_images, distances, x_min, x_max, y_min, y_max)  

    for i in range(len(cropped_images)):
        image = cropped_images[i]
        dist = distances[i]
        xmin = x_min[i]
        xmax = x_max[i]
        ymin = y_min[i]
        ymax = y_max[i]
        box = objects_detected[i]
        print(box)
        object_name = box.name
        center = box.center
        size = box.wlh
        orientation = box.orientation
        euler_orientation = quaternion_to_euler(orientation[0], orientation[1], orientation[2], orientation[3])
        print('center:', center)
        print('size:', size)
        print('euler:', euler_orientation)



        
"""




Module 'NuScenesExplorer' loaded successfully from '/home/nuno120/Documents/SCRIPTIE/scri/lib/python3.10/site-packages/nuscenes/nuscenes.py'
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.406 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


"\nsample_number = 120\n\n# Define the camera names\n#camera_names = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT', 'CAM_BACK_LEFT']\n#camera_names = ['CAM_FRONT_RIGHT']\n\n# Iterate over the camera names\nfor camera_name in camera_names:\n    # Get the data for the sample\n    path, boxes, anns, intrinsic_matrix = extract_bounding_box(sample_number, camera_name)\n    image = cv2.imread(path)\n    print(image.shape)\n    camera_token = nusc.sample[sample_number]['data']['%s' % camera_name]\n    pointsensor_token = nusc.sample[sample_number]['data']['LIDAR_TOP']\n\n    # Get the lidar points and coloring\n    points, coloring, im = map_lidar_to_image(nusc, sample_number, camera_name, pointsensor_token, camera_token)\n\n    #run yolo and show the boxes for comparison\n    #min_x, min_y, max_x, max_y, conf, classes = run_yolo(path, conf_threshold=0.75,visualize=True)\n\n    # Get the training data\n    cropped_images, distances, x_min, x_max, y_min, y_max

In [4]:
import cv2
import multiprocessing
import os

# Define your functions here (extract_bounding_box, map_lidar_to_image, run_yolo, get_training_data, visualize_3DNN, quaternion_to_euler)

def process_sample(sample_number, camera_names):
    #dir = "/home/uva_ki_nunoscholten"
    dir = '/home/nuno120/Documents/SCRIPTIE'
    name = '3DBOXESDIRECTLIDAR'
    directory = os.path.join(dir, name)
    os.makedirs(directory, exist_ok=True)
    
    
    try:
        for camera_name in camera_names:
            # Your existing code here, replacing 'sample_number' with the parameter
            # Remember to collect all the required information into suitable data structures
            
            # Get the data for the sample
            path, boxes, anns, intrinsic_matrix = extract_bounding_box(sample_number, camera_name)
            #image = cv2.imread(path)
            camera_token = nusc.sample[sample_number]['data']['%s' % camera_name]
            pointsensor_token = nusc.sample[sample_number]['data']['LIDAR_TOP']

            # Get the lidar points and coloring
            points, coloring, im = map_lidar_to_image(nusc, sample_number, camera_name, pointsensor_token, camera_token)

            #run yolo and show the boxes for comparison
            #min_x, min_y, max_x, max_y, conf, classes = run_yolo(path, conf_threshold=0.75,visualize=True)

            # Get the training data
            data = get_training_data(im, boxes, anns, points, coloring, intrinsic_matrix)
            #print('datatraining', data)
            cropped_images, image_lidar_coords, distances, x_min, x_max, y_min, y_max, objects_detected = data
    

            # Save images
            for i, cropped_image in enumerate(cropped_images):
                cv2.imwrite(f"{directory}/cropped_image_sample_{sample_number}_camera_{camera_name}_{i}.jpg", cropped_image)
                np.save(f"{directory}/distances_sample_{sample_number}_camera_{camera_name}_{i}.npy", distances[i])
                np.save(f"{directory}/lidarimage_coords_{sample_number}_camera_{camera_name}_{i}.npy", image_lidar_coords[i])
                
            data = []
            for i, box in enumerate(objects_detected):
                size = [x for x in box.wlh]
                center = [x for x in box.center]
                euler_orientation = quaternion_to_euler(box.orientation[0], box.orientation[1], box.orientation[2], box.orientation[3])
                obj_info = {
                    "Object Name": box.name,
                    "Center":  center,
                    "Size": size,
                    "Euler Orientation": euler_orientation,
                    "2D box": [x_min[i], x_max[i], y_min[i], y_max[i]],
                }
                data.append(obj_info)

            
            output_file = f'{directory}/box_info_sample_{sample_number}_camera_{camera_name}.json'

            # Create the directory if it doesn't exist
            os.makedirs(directory, exist_ok=True)

            # Writing the data to a JSON file
            with open(output_file, 'w') as f:
                json.dump(data, f, indent=4)

            """# Write box information to a text file
            with open(f'3DBOXES/box_info_sample_{sample_number}_camera_{camera_name}.txt', 'w') as f:
                for i in range(len(objects_detected)):
                    box = objects_detected[i]
                    f.write(f"Object {i}:\n")
                    f.write(f"Object Name: {box.name}\n")
                    f.write(f"Center: {box.center}\n")
                    f.write(f"Size: {box.wlh}\n")
                    orientation = box.orientation
                    euler_orientation = quaternion_to_euler(orientation[0], orientation[1], orientation[2], orientation[3])
                    f.write(f"Euler Orientation: {euler_orientation}\n")
                    f.write(f"2D box: {x_min[i]}, {x_max[i]}, {y_min[i]}, {y_max[i]}\n")
                    f.write(f"Distances: {distances[i]}\n")
                    f.write("\n")
            """
            print(f"Finished thread{sample_number}")

    except IndexError:
        print(f"IndexError occurred at sample number {sample_number}")

def main():
    camera_names = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT', 'CAM_BACK_LEFT']
    camera_names = ["CAM_FRONT"]
    # Set up your camera names and other parameters
    # camera_names = [...]
    for sample_number in range(400):
        process_sample(sample_number, camera_names)
        print('done,', sample_number)

    """# Set up your multiprocessing
    num_processes = multiprocessing.cpu_count()-5 # You can adjust this as needed
    pool = multiprocessing.Pool(processes=num_processes)

    max_samples =  500000
    # Iterate over samples
    for sample_number in range(max_samples):  # You need to define max_samples
        pool.apply_async(process_sample, args=(sample_number, camera_names))

    pool.close()
    pool.join()"""

if __name__ == "__main__":
    main()


distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distances shape finally,  (64,)
distance

In [ ]:
""